In [4]:
library(shiny)
library(dplyr)
library(ggplot2)
library(DT)


Attachement du package : 'DT'


Les objets suivants sont masqués depuis 'package:shiny':

    dataTableOutput, renderDataTable




In [5]:
# Define server
server <- function(input, output) {
# Réactif pour lire le fichier CSV téléchargé par l'utilisateur
  data <- reactive({
    req(input$file1) # Assure que le fichier est fourni avant de continuer
    read.csv(input$file1$datapath, header = input$header)  # Lire le fichier CSV avec ou sans en-têtes selon l'input
  })
 # Afficher le contenu du DataFrame sous forme de table dans l'interface utilisateur
  output$contents <- renderDT({
    data() # Rendre le DataFrame sous forme de table dynamique
  })
  # Afficher un résumé des données selon le type sélectionné par l'utilisateur
  output$summary <- renderPrint({
    req(input$generate_summary) # Assure que le bouton pour générer le résumé a été cliqué

    df <- data() # Obtenir le DataFrame réactif
# Vérifier le type de résumé sélectionné par l'utilisateur
    if (input$summary_type == "Basic Summary") {
      summary(df) # Afficher un résumé de base des données (moyenne, min, max, etc.)
    } else if (input$summary_type == "Distinct Values") {
      sapply(df, function(x) length(unique(x)))  # Compter le nombre de valeurs distinctes dans chaque colonne
    }
  })
  # Afficher un histogramme de la colonne 'SAR' si elle existe
  output$distPlot <- renderPlot({
    req(data())  # Assure que le DataFrame est disponible avant de créer le graphique

    df <- data() # Obtenir le DataFrame réactif
# Vérifier si la colonne 'SAR' existe dans le DataFrame
    if ("SAR" %in% colnames(df)) {
      ggplot(df, aes(x = SAR)) + # Créer un histogramme pour la colonne 'SAR'
        geom_histogram(binwidth = 10000, fill = "green", color = "black") + # Définir les paramètres du graphique
        labs(title = "Distribution of SAR", x = "SAR", y = "Count") # Ajouter des labels
    } else {
      plot.new() # Créer une nouvelle zone de graphique vide
      text(0.5, 0.5, "SAR column not found in the data", cex = 1.5) # Afficher un message d'erreur si la colonne 'SAR' n'est pas trouvée
    }
  })
}

In [6]:
# Define UI
ui <- fluidPage(
# Titre de l'application affiché en haut de la page
  titlePanel("Data Ingestion and Summary"),
# Mise en page avec une barre latérale pour les contrôles et un panneau principal pour les sorties
  sidebarLayout(
    # Panneau latéral pour les entrées de l'utilisateur
    sidebarPanel(
    # Input pour choisir un fichier CSV à télécharger
      fileInput("file1", "Choose CSV File", accept = ".csv"),
    # Case à cocher pour indiquer si le fichier CSV contient des en-têtes
      checkboxInput("header", "Header", TRUE),
    # Bouton pour générer le résumé des données
      actionButton("generate_summary", "Generate Summary"),
    # Menu déroulant pour sélectionner le type de résumé
      selectInput("summary_type", "Summary Type", choices = c("Basic Summary", "Distinct Values"))
    ),
     # Panneau principal pour afficher les résultats
    mainPanel(
    # Affichage du contenu du fichier CSV sous forme de table interactive
      DTOutput("contents"),
    # Affichage du résumé des données (texte brut)
      verbatimTextOutput("summary"),
    # Affichage d'un graphique (histogramme)
      plotOutput("distPlot")
    )
  )
)


In [ ]:
# Run the application
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:3154

